# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3_ver3/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    #try:
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    gmm_param = history['gmm_param']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    gmm_param_path = MODEL_DIR+'/{}_gmm_param.pkl'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    pd.to_pickle(gmm_param, gmm_param_path)
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    #except:
    #    print("error")
    #    history = "error"
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types[4:]:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-12-02 02:32:39,863 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyConveyor
2020-12-02 02:32:39,864 - 00_train.py - INFO - MAKE DATA_LOADER
2020-12-02 02:32:40,758 - 00_train.py - INFO - TRAINING
  0%|          | 0/47 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 27/27 [00:03<00:00,  7.66it/s]
2020-12-02 02:33:11,343 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:27009.830444, tr_rec:24930.160278, tr_eng:-431.106297, tr_covd:424556.173340, val_AUC:0.500713, val_pAUC:0.507132
100%|██████████| 27/27 [00:03<00:00,  7.44it/s]
2020-12-02 02:33:40,788 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:26432.601379, tr_rec:24603.091125, tr_eng:-476.492180, tr_covd:375431.830078, val_AUC:0.497080, val_pAUC:0.492639
100%|██████████| 27/27 [00:03<00:00,  7.46it/s]
2020-12-02 02:34:09,953 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:25877.025208, tr_rec:24199.489288, tr_eng:-486.602209, tr_covd:345239.263672, val_AUC:0.480829, val_pAUC:0.487503
100%|██████████| 27/27 [00:03<00:00,  7.59it/s]
2020-12-02 02:34:39,188 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:25396.791992, tr_rec:23805.337433, tr_eng:-492.574502, tr_covd:328142.388672, val_AUC:0.472583, val_pAUC:0.482423
100%|██████████| 27/27 [00:03<00:00,  7.48it

100%|██████████| 27/27 [00:03<00:00,  7.54it/s]
2020-12-02 02:47:26,893 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:1299.593105, tr_rec:745.113103, tr_eng:-468.028763, tr_covd:120256.580200, val_AUC:0.499888, val_pAUC:0.496221
100%|██████████| 27/27 [00:03<00:00,  7.56it/s]
2020-12-02 02:47:56,264 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:1314.036512, tr_rec:748.295371, tr_eng:-487.787777, tr_covd:122903.987061, val_AUC:0.518892, val_pAUC:0.496009
100%|██████████| 27/27 [00:03<00:00,  7.38it/s]
2020-12-02 02:48:26,179 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:1242.605368, tr_rec:723.451910, tr_eng:-537.228264, tr_covd:114575.258911, val_AUC:0.519545, val_pAUC:0.497782
100%|██████████| 27/27 [00:03<00:00,  7.75it/s]
2020-12-02 02:48:55,373 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:1176.482494, tr_rec:726.102670, tr_eng:-538.045353, tr_covd:100836.875244, val_AUC:0.504059, val_pAUC:0.501666
100%|██████████| 27/27 [00:03<00:00,  7.67it/s]
2020

100%|██████████| 27/27 [00:03<00:00,  7.56it/s]
2020-12-02 03:01:43,780 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:915.869469, tr_rec:649.430633, tr_eng:-620.351723, tr_covd:65694.801514, val_AUC:0.509590, val_pAUC:0.498065
100%|██████████| 27/27 [00:03<00:00,  7.57it/s]
2020-12-02 03:02:13,087 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:918.466618, tr_rec:649.750705, tr_eng:-616.151966, tr_covd:66066.223145, val_AUC:0.477044, val_pAUC:0.496745
100%|██████████| 27/27 [00:03<00:00,  7.77it/s]
2020-12-02 03:02:42,193 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:905.876770, tr_rec:652.395633, tr_eng:-630.873427, tr_covd:63313.697510, val_AUC:0.541579, val_pAUC:0.510931
100%|██████████| 27/27 [00:03<00:00,  7.52it/s]
2020-12-02 03:03:11,637 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:928.229004, tr_rec:659.513116, tr_eng:-611.691319, tr_covd:65977.005005, val_AUC:0.520759, val_pAUC:0.495856
100%|██████████| 27/27 [00:03<00:00,  7.48it/s]
2020-12-02 0

...

100%|██████████| 7/7 [00:01<00:00,  5.34it/s]
2020-12-02 07:26:21,704 - pytorch_modeler.py - INFO - Epoch 204/300:train_loss:911.781935, tr_rec:735.683283, tr_eng:-594.071433, tr_covd:47101.160583, val_AUC:0.492736, val_pAUC:0.521168
100%|██████████| 7/7 [00:01<00:00,  5.17it/s]
2020-12-02 07:26:55,803 - pytorch_modeler.py - INFO - Epoch 205/300:train_loss:903.964577, tr_rec:732.265487, tr_eng:-598.131811, tr_covd:46302.454102, val_AUC:0.522698, val_pAUC:0.514864
100%|██████████| 7/7 [00:01<00:00,  4.88it/s]
2020-12-02 07:27:29,770 - pytorch_modeler.py - INFO - Epoch 206/300:train_loss:909.488722, tr_rec:737.514503, tr_eng:-609.684467, tr_covd:46588.534424, val_AUC:0.568137, val_pAUC:0.544737
100%|██████████| 7/7 [00:01<00:00,  4.70it/s]
2020-12-02 07:28:03,867 - pytorch_modeler.py - INFO - Epoch 207/300:train_loss:908.270327, tr_rec:735.846848, tr_eng:-613.188290, tr_covd:46748.464111, val_AUC:0.619459, val_pAUC:0.574132
100%|██████████| 7/7 [00:01<00:00,  4.90it/s]
2020-12-02 07:28:3

100%|██████████| 7/7 [00:01<00:00,  4.58it/s]
2020-12-02 07:42:47,403 - pytorch_modeler.py - INFO - Epoch 233/300:train_loss:897.208351, tr_rec:728.632359, tr_eng:-629.534664, tr_covd:46305.891113, val_AUC:0.468209, val_pAUC:0.555379
100%|██████████| 7/7 [00:01<00:00,  4.77it/s]
2020-12-02 07:43:21,493 - pytorch_modeler.py - INFO - Epoch 234/300:train_loss:896.570496, tr_rec:727.775666, tr_eng:-628.783062, tr_covd:46334.626404, val_AUC:0.511091, val_pAUC:0.578355
100%|██████████| 7/7 [00:01<00:00,  5.02it/s]
2020-12-02 07:43:55,144 - pytorch_modeler.py - INFO - Epoch 235/300:train_loss:895.534975, tr_rec:729.438076, tr_eng:-638.474801, tr_covd:45988.876282, val_AUC:0.424099, val_pAUC:0.538982
100%|██████████| 7/7 [00:01<00:00,  4.97it/s]
2020-12-02 07:44:29,202 - pytorch_modeler.py - INFO - Epoch 236/300:train_loss:892.541893, tr_rec:724.226128, tr_eng:-623.612231, tr_covd:46135.399902, val_AUC:0.547291, val_pAUC:0.575564
100%|██████████| 7/7 [00:01<00:00,  4.93it/s]
2020-12-02 07:45:0

100%|██████████| 7/7 [00:01<00:00,  5.17it/s]
2020-12-02 07:59:12,658 - pytorch_modeler.py - INFO - Epoch 262/300:train_loss:892.274448, tr_rec:724.514978, tr_eng:-631.104592, tr_covd:46173.987000, val_AUC:0.667192, val_pAUC:0.553268
100%|██████████| 7/7 [00:01<00:00,  5.18it/s]
2020-12-02 07:59:46,794 - pytorch_modeler.py - INFO - Epoch 263/300:train_loss:889.543291, tr_rec:723.518551, tr_eng:-633.881810, tr_covd:45882.585815, val_AUC:0.573676, val_pAUC:0.562984
100%|██████████| 7/7 [00:01<00:00,  4.87it/s]
2020-12-02 08:00:21,118 - pytorch_modeler.py - INFO - Epoch 264/300:train_loss:890.882874, tr_rec:725.725801, tr_eng:-640.083560, tr_covd:45833.086243, val_AUC:0.545511, val_pAUC:0.552429
100%|██████████| 7/7 [00:01<00:00,  5.19it/s]
2020-12-02 08:00:54,982 - pytorch_modeler.py - INFO - Epoch 265/300:train_loss:887.910742, tr_rec:723.664296, tr_eng:-642.140000, tr_covd:45692.088867, val_AUC:0.617591, val_pAUC:0.542872
100%|██████████| 7/7 [00:01<00:00,  5.22it/s]
2020-12-02 08:01:2

100%|██████████| 7/7 [00:01<00:00,  4.70it/s]
2020-12-02 08:15:39,999 - pytorch_modeler.py - INFO - Epoch 291/300:train_loss:890.655302, tr_rec:722.946509, tr_eng:-630.401947, tr_covd:46149.797852, val_AUC:0.530206, val_pAUC:0.527935
100%|██████████| 7/7 [00:01<00:00,  4.78it/s]
2020-12-02 08:16:13,809 - pytorch_modeler.py - INFO - Epoch 292/300:train_loss:885.261400, tr_rec:721.177107, tr_eng:-640.010588, tr_covd:45617.069885, val_AUC:0.571250, val_pAUC:0.572368
100%|██████████| 7/7 [00:01<00:00,  5.23it/s]
2020-12-02 08:16:47,807 - pytorch_modeler.py - INFO - Epoch 293/300:train_loss:888.775066, tr_rec:722.304573, tr_eng:-634.725854, tr_covd:45988.616089, val_AUC:0.623736, val_pAUC:0.533516
100%|██████████| 7/7 [00:01<00:00,  4.99it/s]
2020-12-02 08:17:21,895 - pytorch_modeler.py - INFO - Epoch 294/300:train_loss:888.672707, tr_rec:723.164723, tr_eng:-635.488411, tr_covd:45811.363647, val_AUC:0.563560, val_pAUC:0.528282
100%|██████████| 7/7 [00:01<00:00,  5.12it/s]
2020-12-02 08:17:5

elapsed time: 10274.277701139 [sec]


history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)